## The `chatlas` package for a consistent LLM interface and workflow

One of the frustrations of working with different LLM providers is the difference in API structures.  This has historically meant that developers have had to code up their LLM workflows quite differently if they were working with, say, OpenAI versus Anthropic or Llama.

The `chatlas` package attempts to overcome this by offering a set of classes and methods that have greater alignment across LLM providers.  This means that starting a chat, or using tool calling or other services will look the same or very similar, no matter which LLM provider you are using.  For those familiar, `chatlas` is basically the Python equivalent of R's `ellmer` package.

The following is a short illustratory demo of some of the features of `chatlas`.

### Starting a chat session - OpenAI example

In [1]:
# import packages and key environment variables
from dotenv import load_dotenv
from chatlas import ChatOpenAI, ChatAnthropic, ChatOllama
import os

load_dotenv()
INSTANCE_ID = os.getenv('INSTANCE_ID')
API_KEY = os.getenv('API_KEY')
BASE_URL_STEM = os.getenv('BASE_URL_STEM')

In [2]:
# set up an openai chat client
PROVIDER = "openai"
BASE_URL = f"https://{PROVIDER}.{BASE_URL_STEM}/{INSTANCE_ID}/v1"

chat = ChatOpenAI(
    model = "gpt-4o",
    api_key = API_KEY,
    base_url = BASE_URL,
    system_prompt = "You are a friendly but terse assistant.",
)

In [ ]:
# start a chat app
chat.app()

In [ ]:
# chat in the console
chat.console()

In [3]:
# programmatic chat
chat.chat("What exactly is a spirit vegetable?")

<br>

A spirit vegetable is a concept similar to a "spirit animal," but instead of representing a guiding animal, it represents a vegetable that embodies aspects of your personality or traits you identify with. It's often used humorously or metaphorically to showcase personal characteristics or preferences.

### Tool (function) calling - OpenAI example

In [4]:
# tool (function) calling - function to get current temperature
import requests

# function to get the current temperature in a place
def get_current_temperature(place: str) -> str:
  """Get the current temperature in a given place."""
  base_url = f"https://wttr.in/{place}?format=j1"
  response = requests.get(base_url)
  data = response.json()
  return f"The current temperature in {place} is {data['current_condition'][0]['temp_C']} degrees Celsius"

# test the function
get_current_temperature("London")

'The current temperature in London is 13 degrees Celsius'

In [5]:
# register the function with your chat
chat.register_tool(get_current_temperature)

In [6]:
# now the chat can use it
chat.chat("I'm in Atlanta today and I'm told I should wear warm clothes.  What do you think?")

<br>



```python
# 🔧 tool request (call_NGnedemBOwe2F3VRyIPgNHRE)
get_current_temperature(place=Atlanta)
```




```python
# ✅ tool result (call_NGnedemBOwe2F3VRyIPgNHRE)
The current temperature in Atlanta is 22 degrees Celsius
```

<br>

The current temperature in Atlanta is 22 degrees Celsius (about 72 degrees Fahrenheit). That's quite mild, so you might not need very warm clothes. Dressing in layers could be a comfortable choice.

### Structured data extraction - OpenAI example

In [7]:
# pulling structured data out of text
from pydantic import BaseModel

class Person(BaseModel):
    name: str
    pets: int
    skills: list[str]

chat.extract_data(
  "My name is Keith.  I have two cats and one dog named Bertie.  I am very good at Math and Computer Games", 
  data_model=Person,
)

{'name': 'Keith', 'pets': 3, 'skills': ['Math', 'Computer Games']}

### Start a new chat - Anthropic example

In [8]:
# now try a Anthropic chat client- note similar but not 100% identical to OpenAI
PROVIDER = "anthropic"
BASE_URL = f"https://{PROVIDER}.{BASE_URL_STEM}/{INSTANCE_ID}"

chat = ChatAnthropic(
    api_key = API_KEY,
    system_prompt = "You are a friendly but terse assistant.",
    kwargs = {"base_url": BASE_URL}  
)

In [9]:
# programmatic chat
chat.chat("Which integer is commonly quoted as the answer to the meaning of life?")

<br>

42, from "The Hitchhiker's Guide to the Galaxy" by Douglas Adams.

In [10]:
# register tool
chat.register_tool(get_current_temperature)

In [11]:
# check tool use
chat.chat("My sister is heading to the capital of Norway next week?  How should she pack?")

<br>

Your sister is heading to Oslo, the capital of Norway. To advise on packing, I should check the current temperature there.

```python
# 🔧 tool request (toolu_016VfaGGMyKQ7hZJkq4KTvcS)
get_current_temperature(place=Oslo, Norway)
```




```python
# ✅ tool result (toolu_016VfaGGMyKQ7hZJkq4KTvcS)
The current temperature in Oslo, Norway is 19 degrees Celsius
```

<br>

Based on the current temperature of 19°C (66°F) in Oslo, your sister should pack for mild weather. I'd suggest:

- Light to medium jackets or sweaters
- Mix of short and long sleeve shirts for layering
- Comfortable walking shoes
- A light rain jacket (Norway can be rainy)
- Standard travel essentials

The weather is relatively pleasant now, but she should check the forecast closer to her trip as conditions in Norway can change.

In [12]:
#  extract structured data
class Person(BaseModel):
    name: str
    pets: int
    areas_of_expertise: list[str]
    qualifications: list[str]

chat.extract_data(
  """
  My name is Keith.  I had two cats and one dog named Bertie, but I recently sold the two cats.  I have a PhD in Pure Mathematics, and I am also a holder of the Licentiate Performing Diploma from Trinity College of Music in London, taking my final exams in the Recorder.
  """, 
  data_model = Person,
)

{'name': 'Keith',
 'pets': 1,
 'areas_of_expertise': ['Pure Mathematics', 'Music', 'Recorder'],
 'qualifications': ['PhD in Pure Mathematics',
  'Licentiate Performing Diploma from Trinity College of Music']}

In [13]:
# update structured information during chat session
chat.extract_data(
  """
  Oh it's Keith again.  I'm sorry, I'm so dumb.  I forgot to mention I also have a new puppy which we only just picked up this week.
  """, 
  data_model = Person,
)

{'name': 'Keith',
 'pets': 2,
 'areas_of_expertise': ['Pure Mathematics', 'Music', 'Recorder'],
 'qualifications': ['PhD in Pure Mathematics',
  'Licentiate Performing Diploma from Trinity College of Music']}

In [14]:
# update again
chat.extract_data(
  """
  It's Keith again.  I just spoke to my wife and she left the door open and the puppy ran out on the street and got killed.  It's a bad day.
  """, 
  data_model = Person,
)

{'name': 'Keith',
 'pets': 1,
 'areas_of_expertise': ['Pure Mathematics', 'Music', 'Recorder'],
 'qualifications': ['PhD in Pure Mathematics',
  'Licentiate Performing Diploma from Trinity College of Music']}

### New chat session with local model using `ollama`

In [15]:
# local ollama model
chat = ChatOllama(
    model = "llama3.1:8b",
    system_prompt = "You are a friendly but terse assistant.",
)

In [16]:
# programmatic chat
chat.chat("Explain Buddhism in two sentences")

<br>

Buddhism is a spiritual path that aims to end suffering by understanding and letting go of the causes of our pain – typically through recognizing the impermanence and illusiveness of all things, including ourselves. The ultimate goal is liberation from this cycle, known as Nirvana, achieved by following the Eightfold Path and cultivating mindfulness and compassion.

In [17]:
# extract data using earlier data model
chat.extract_data(
      """
  My name is Keith.  I had two cats and one dog named Bertie, but I recently sold the two cats.  I have a PhD in Pure Mathematics, and I am also a holder of the Licentiate Performing Diploma from Trinity College of Music in London, taking my final exams in the Recorder.
  """, 
  data_model = Person,
)

{'name': 'Keith',
 'pets': 1,
 'areas_of_expertise': ['Pure mathematics'],
 'qualifications': ['PhD, Music']}

In [18]:
# register temperature tool
chat.register_tool(get_current_temperature)

In [19]:
# use temperature tool
chat.chat("I'm heading to Tasmania tomorrow but I forgot my sunscreen.  Should I be concerned?")

<br>



```python
# 🔧 tool request (call_imi9ak7k)
get_current_temperature(place=Tasmania)
```




```python
# ✅ tool result (call_imi9ak7k)
The current temperature in Tasmania is 8 degrees Celsius
```

<br>

You still shouldn't be too concerned about the sunscreen, although it's a good reminder to pack some. However, you might want to prioritize packing warm clothing for your visit to Tasmania instead - it can get quite chilly there!